In [1]:
import numpy as np
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 101)

In [3]:
# Load joined dataframe
df = pd.read_csv('csv/metadata_text_merged_cleaned.csv')

# Frequency analysis

In [4]:
# Get list of words that appear most frequently
freq = pd.Series(' '.join(df['text_cleaned']).split()).value_counts()
dealiased_freq = pd.Series(' '.join(df['text_cleaned_dealiased']).split()).value_counts()

In [5]:
print 'Before:', len(freq), 'After:', len(dealiased_freq)

Before: 17001 After: 15625


In [6]:
freq[:100]

send            2713
shall           2590
sent            2585
mr              2398
one             2353
would           1927
received        1863
two             1833
goods           1711
slaves          1674
per             1660
desire          1651
canoe           1618
captain         1615
may             1560
soe             1342
last            1243
come            1222
wee             1215
great           1214
worship         1208
company         1140
order           1119
much            1112
doe             1107
men             1106
good            1078
att             1044
give            1018
people          1003
came            1000
time             997
corne            940
pleased          921
royall           912
accompt          909
itt              908
hope             899
noe              886
downe            882
dutch            876
hand             876
instant          871
hee              857
also             854
hath             851
mee              838
since        

In [7]:
dealiased_freq[:100]

send           2713
sent           2592
shall          2590
mr             2398
one            2353
would          1927
received       1866
two            1833
canoe          1752
goods          1711
slaves         1674
per            1660
account        1652
desire         1651
captain        1621
may            1560
great          1366
so             1342
worship        1294
corn           1243
last           1243
come           1222
we             1215
also           1177
company        1155
order          1119
much           1112
do             1107
men            1106
good           1078
time           1070
at             1044
give           1018
people         1003
came           1000
royal           951
worships        948
pleased         921
it              908
dutch           904
hope            899
coast           891
no              886
down            882
cape            881
hand            876
instant         871
he              857
hath            851
castle          850
